# Multimodal 3D imaging + tabular model  
*K-fold training → per-fold Isotonic calibration → independent TEST evaluation*

This notebook contains an end-to-end pipeline used in a medical imaging research project:

1. **K-fold training** of a multimodal model (tabular + 3D-CNN).
2. Saving **all epoch checkpoints** and a `best_models.pkl` file (best epoch per fold by validation AUC).
3. **Isotonic Regression calibration** (per fold) and **TEST evaluation** with:
   - ROC curve + AUC **bootstrap 95% CI**
   - Calibration curve (reliability plot)
   - Decision Curve Analysis (net benefit)

## Quick start
1. Edit `data_path_aug` and `checkpoint_folder` in **“Imports & Global Settings”**.
2. Run cells from top to bottom.
3. Train folds (**Fold 0–4**) by running the fold cells you need. You can stop/restart at any time; the best model paths are tracked in `best_models.pkl`.
4. Run the final evaluation cell to generate metrics and plots.

## Expected dataset format (`.npz`)
The dataset file must include these arrays (names are required):

| key | shape (example) | dtype | description |
|---|---:|---|---|
| `x_train_val_image` | `(N_train, D, H, W, 1)` or `(N_train, D, H, W)` | float32 | 3D volumes (channel-last) |
| `x_train_val_table` | `(N_train, F)` | float32 | tabular features |
| `y_train_val` | `(N_train,)` | int/bool | binary labels |
| `x_test_image` | `(N_test, D, H, W, 1)` or `(N_test, D, H, W)` | float32 | test volumes |
| `x_test_table` | `(N_test, F)` | float32 | test features |
| `y_test` | `(N_test,)` | int/bool | test labels |

If your image arrays do not include a channel dimension, the notebook will add `[..., None]`.

## Note on probability calibration
In this notebook, the Isotonic model is fit on each fold’s **validation** predictions and applied to:
- that fold’s validation set (to build *OOF calibrated* probabilities), and
- the test set (to build a calibrated ensemble).

For strictly unbiased calibration (especially if calibration itself is a primary endpoint), consider **nested CV** or fitting calibration on an inner split only.


## Environment (optional)

This notebook can run locally or on a GPU-enabled environment.

Recommended packages:

- Python ≥ 3.10
- TensorFlow / Keras (GPU recommended for training; **tested with `keras==3.8.0`**)
- NumPy, SciPy, scikit-learn, Matplotlib

In [ ]:
# If you see version conflicts, uncomment the line below, run the cell, then restart the runtime.
# !pip install keras==3.8.0

## Configuration

Edit the paths and key hyperparameters in the next code cell (**Imports & Global Settings**):

- `data_path_aug` : path to the `.npz` file (train/val + test)
- `checkpoint_folder` : output directory for checkpoints and `best_models.pkl`
- `NUM_FOLDS`, `EPOCHS`, `BATCH_SIZE`, `LR` : training settings


In [ ]:
# =========================================================
# 0) Imports & Global Settings
# =========================================================
import os
import re
import glob
import time
import gc
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

from scipy.stats import beta

from sklearn.model_selection import KFold
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import calibration_curve
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve

# -------------------------
# Reproducibility
# -------------------------
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# -------------------------
# Training settings
# -------------------------
BATCH_SIZE = 2
NUM_FOLDS  = 5
EPOCHS     = 50
LR         = 5e-4

# -------------------------
# Calibration / evaluation settings
# -------------------------
CALIB_N_BINS   = 5
CALIB_STRATEGY = "quantile"   # "uniform" or "quantile"

CI_ALPHA   = 0.05
AUC_N_BOOT = 2000

# -------------------------
# Paths (EDIT HERE)
# -------------------------
# You can either:
#   (A) set environment variables `DATA_PATH_AUG` and `CHECKPOINT_DIR`, or
#   (B) edit the default values below.
#
# Expected file: a single `.npz` containing train/val + test arrays (see the first markdown cell).
data_path_aug = os.environ.get("DATA_PATH_AUG", "./data/dataset_augmented.npz")

# Where to write checkpoints (`.keras`) and `best_models.pkl`
checkpoint_folder = os.environ.get("CHECKPOINT_DIR", "./checkpoints")

best_models_file = os.path.join(checkpoint_folder, "best_models.pkl")
os.makedirs(checkpoint_folder, exist_ok=True)

print("Settings loaded.")
print("Python:", __import__("sys").version.split()[0])
print("TensorFlow:", tf.__version__)
try:
    import keras as keras_pkg  # standalone package (optional)
    print("keras package:", getattr(keras_pkg, "__version__", "unknown"))
except Exception:
    pass
print("checkpoint_folder:", checkpoint_folder)
print("data_path_aug:", data_path_aug)


In [ ]:
# =========================================================
# 2) Load data (augmented train/val + test)
#    - IMPORTANT: We shuffle train/val ONCE with SEED and reuse the same order
#      for both training (fold splits) and evaluation.
# =========================================================
if not os.path.exists(data_path_aug):
    raise FileNotFoundError(f"Augmented data file not found: {data_path_aug}")

loaded_aug = np.load(data_path_aug)

x_train_val_image = loaded_aug["x_train_val_image"]
x_train_val_table = loaded_aug["x_train_val_table"]
y_train_val       = loaded_aug["y_train_val"].astype(int).ravel()

x_test_image = loaded_aug["x_test_image"]
x_test_table = loaded_aug["x_test_table"]
y_test       = loaded_aug["y_test"].astype(int).ravel()

print("Loaded augmented data.")
print("Train/Val:", x_train_val_image.shape, x_train_val_table.shape, y_train_val.shape)
print("Test     :", x_test_image.shape, x_test_table.shape, y_test.shape)

# --- shuffle train/val deterministically ---
idx = np.arange(len(y_train_val))
rng = np.random.default_rng(SEED)
rng.shuffle(idx)

x_train_val_image = x_train_val_image[idx]
x_train_val_table = x_train_val_table[idx]
y_train_val       = y_train_val[idx]

print("After deterministic shuffle (train/val):", x_train_val_image.shape, x_train_val_table.shape, y_train_val.shape)


# Shapes
N_FEATURES = x_train_val_table.shape[1]
IMG_SHAPE  = (x_train_val_image.shape[1], x_train_val_image.shape[2], x_train_val_image.shape[3], 1)

print("N_FEATURES:", N_FEATURES)
print("IMG_SHAPE :", IMG_SHAPE)
# --- ensure expected dtypes/shapes ---
# The training pipeline expects float inputs and channel-last images.
x_train_val_image = x_train_val_image.astype(np.float32)
x_train_val_table = x_train_val_table.astype(np.float32)
x_test_image      = x_test_image.astype(np.float32)
x_test_table      = x_test_table.astype(np.float32)

# Add channel dimension if missing: (N, D, H, W) -> (N, D, H, W, 1)
if x_train_val_image.ndim == 4:
    x_train_val_image = x_train_val_image[..., None]
if x_test_image.ndim == 4:
    x_test_image = x_test_image[..., None]


## Model definitions (tabular + 3D-CNN + multimodal)

This section defines a simple reference architecture:
- **Tabular branch**: currently an identity/pass-through (edit to add an MLP if needed).
- **Image branch**: a lightweight 3D-CNN that outputs a 1-unit sigmoid probability.
- **Multimodal head**: concatenates tabular features and the image probability, then outputs a final sigmoid.

Feel free to adjust filters, depth, dropout, and the tabular sub-network to match your task and compute budget.


In [ ]:
# =========================================================
# 3) Model definitions
#    - IMPORTANT: We build a *fresh* model per fold to avoid weight sharing across folds.
# =========================================================

def model_tabular(n_features: int, fold: int):
    # Simple NN for tabular input (postoperative day etc.)
    inputs = keras.Input((n_features,), name=f"tabular_input_fold_{fold}")
    x = inputs  # identity (you can add Dense layers here if needed)
    return keras.Model(inputs, x, name=f"TabularModel_Fold_{fold}")

def model_image(img_shape, fold: int):
    # 3D CNN for image input
    inputs = keras.Input(img_shape, name=f"image_input_fold_{fold}")

    x = layers.Conv3D(64, 3, kernel_initializer="he_normal", name=f"conv3d_1_fold_{fold}")(inputs)
    x = layers.BatchNormalization(name=f"bn_1_fold_{fold}")(x)
    x = layers.Activation("relu", name=f"relu_1_fold_{fold}")(x)
    x = layers.MaxPool3D(2, name=f"pool_1_fold_{fold}")(x)

    x = layers.Conv3D(64, 3, kernel_initializer="he_normal", name=f"conv3d_2_fold_{fold}")(x)
    x = layers.BatchNormalization(name=f"bn_2_fold_{fold}")(x)
    x = layers.Activation("relu", name=f"relu_2_fold_{fold}")(x)
    x = layers.MaxPool3D(2, name=f"pool_2_fold_{fold}")(x)

    x = layers.Conv3D(128, 3, kernel_initializer="he_normal", name=f"conv3d_3_fold_{fold}")(x)
    x = layers.BatchNormalization(name=f"bn_3_fold_{fold}")(x)
    x = layers.Activation("relu", name=f"relu_3_fold_{fold}")(x)
    x = layers.MaxPool3D(2, name=f"pool_3_fold_{fold}")(x)

    x = layers.Conv3D(256, 3, kernel_initializer="he_normal", name=f"conv3d_4_fold_{fold}")(x)
    x = layers.BatchNormalization(name=f"bn_4_fold_{fold}")(x)
    x = layers.Activation("relu", name=f"relu_4_fold_{fold}")(x)
    x = layers.MaxPool3D(2, name=f"pool_4_fold_{fold}")(x)

    x = layers.GlobalAveragePooling3D(name=f"gap_fold_{fold}")(x)
    x = layers.Dense(512, activation="relu", kernel_initializer="he_normal", name=f"dense_1_fold_{fold}")(x)
    x = layers.Dropout(0.3, name=f"dropout_1_fold_{fold}")(x)

    # Keep sigmoid head as in the original notebook (used as a feature in multimodal)
    outputs = layers.Dense(1, activation="sigmoid", name=f"image_prob_fold_{fold}")(x)
    return keras.Model(inputs, outputs, name=f"ImageModel_Fold_{fold}")

def multimodal_model(n_features: int, img_shape, fold: int):
    # Combine tabular + image branches
    tab_m = model_tabular(n_features, fold)
    img_m = model_image(img_shape, fold)

    combined = layers.Concatenate(name=f"concat_fold_{fold}")([tab_m.output, img_m.output])
    outputs  = layers.Dense(1, activation="sigmoid", name=f"output_fold_{fold}")(combined)

    model = keras.Model(inputs=[tab_m.input, img_m.input], outputs=outputs, name=f"MultimodalModel_Fold_{fold}")
    return model

# quick sanity check (fold 0)
tmp = multimodal_model(N_FEATURES, IMG_SHAPE, fold=0)
tmp.summary()
del tmp
gc.collect()


## Training utilities + KFold splits (no training loop)

We create the `KFold` splits **once** and reuse the indices for both training and evaluation.

**Why each fold is a separate cell** (instead of a single for-loop):
- easier to resume after interruptions (common on Colab)
- less GPU memory fragmentation
- lets you inspect training curves per fold

During training, the notebook saves **all epoch checkpoints** (`fold_{k}_{epoch}.keras`).
The best epoch is selected by **validation AUC** and its path is stored in `best_models.pkl`.


In [ ]:
# =========================================================
# 4) Training utilities (tf.data) + deterministic fold splits
# =========================================================

def plot_fold_history(history, fold):
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    for i, metric in enumerate(["loss", "auc"]):
        ax[i].plot(history.history.get(metric, []), label="Train")
        ax[i].plot(history.history.get("val_" + metric, []), label="Validation")
        ax[i].set_title(f"Fold {fold} - {metric.upper()}")
        ax[i].set_xlabel("Epochs")
        ax[i].set_ylabel(metric.upper())
        ax[i].legend()
    plt.tight_layout()
    plt.show()

def data_generator(x_table, x_image, y, batch_size):
    while True:
        for start in range(0, len(y), batch_size):
            end = start + batch_size
            yield ((x_table[start:end], x_image[start:end]), y[start:end])

def create_dataset(x_table, x_image, y, batch_size, shuffle=False, seed=SEED):
    ds = tf.data.Dataset.from_generator(
        lambda: data_generator(x_table, x_image, y, batch_size),
        output_signature=(
            (
                tf.TensorSpec(shape=(None, x_table.shape[1]), dtype=tf.float32),
                tf.TensorSpec(shape=(None, x_image.shape[1], x_image.shape[2], x_image.shape[3], 1), dtype=tf.float32),
            ),
            tf.TensorSpec(shape=(None,), dtype=tf.float32),
        ),
    )
    if shuffle:
        ds = ds.shuffle(buffer_size=min(len(y), 2048), seed=seed, reshuffle_each_iteration=True)
    return ds

# Prepare fold splits ONCE and reuse
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=SEED)
folds = list(kf.split(x_train_val_table))
print(f"Prepared folds: {len(folds)} splits")


In [ ]:
# =========================================================
# 5) Train ONE fold (call this from the fold-specific cells below)
# =========================================================

def _load_best_models_list(path: str, num_folds: int):
    if os.path.exists(path):
        with open(path, "rb") as f:
            bm = pickle.load(f)
        # Accept list or dict
        if isinstance(bm, dict):
            return [bm.get(i, None) for i in range(num_folds)]
        if isinstance(bm, list):
            if len(bm) < num_folds:
                bm = bm + [None] * (num_folds - len(bm))
            return bm[:num_folds]
    return [None] * num_folds

def _save_best_models_list(path: str, best_models_list):
    with open(path, "wb") as f:
        pickle.dump(best_models_list, f)

def train_one_fold(fold: int,
                   epochs: int = EPOCHS,
                   batch_size: int = BATCH_SIZE,
                   lr: float = LR,
                   steps_per_epoch_min: int = 1):
    print(f"\n===== Training fold {fold}/{NUM_FOLDS-1} =====")

    train_idx, val_idx = folds[fold]
    x_tr_tab, x_va_tab = x_train_val_table[train_idx], x_train_val_table[val_idx]
    x_tr_img, x_va_img = x_train_val_image[train_idx], x_train_val_image[val_idx]
    y_tr, y_va         = y_train_val[train_idx], y_train_val[val_idx]

    # tf.data datasets
    train_ds = create_dataset(x_tr_tab, x_tr_img, y_tr, batch_size, shuffle=True, seed=SEED).prefetch(tf.data.AUTOTUNE)
    val_ds   = create_dataset(x_va_tab, x_va_img, y_va, batch_size, shuffle=False).prefetch(tf.data.AUTOTUNE)

    # Reset graph / free memory
    tf.keras.backend.clear_session()
    gc.collect()

    # Build a fresh model for this fold
    model = multimodal_model(N_FEATURES, IMG_SHAPE, fold=fold)
    model.compile(
        loss="binary_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        metrics=[
            tf.keras.metrics.AUC(name="auc"),
            tf.keras.metrics.Recall(name="recall"),
        ],
    )

    ckpt_pattern = os.path.join(checkpoint_folder, f"fold_{fold}_{{epoch:03d}}.keras")
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath=ckpt_pattern,
        save_best_only=False,   # keep same behavior as original
        verbose=0,
    )

    steps_per_epoch = max(steps_per_epoch_min, len(y_tr) // batch_size)
    val_steps       = max(1, len(y_va) // batch_size)

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        validation_steps=val_steps,
        callbacks=[checkpoint_cb],
        verbose=1,
    )

    best_epoch = int(np.argmax(history.history["val_auc"]) + 1)
    best_auc   = float(np.max(history.history["val_auc"]))
    best_model_path = os.path.join(checkpoint_folder, f"fold_{fold}_{best_epoch:03d}.keras")

    # Save/update best_models.pkl (index = fold)
    best_models = _load_best_models_list(best_models_file, NUM_FOLDS)
    best_models[fold] = best_model_path
    _save_best_models_list(best_models_file, best_models)

    print(f"Best validation AUC for fold {fold}: {best_auc:.4f} at epoch {best_epoch}")
    print("Best model path:", best_model_path)
    print("Updated:", best_models_file)

    plot_fold_history(history, fold)

    return history


### Train fold 0

Run this cell to train **fold 0**.

Tips:
- Adjust `EPOCHS`, `BATCH_SIZE`, and `LR` in **Imports & Global Settings**.
- The notebook saves all epoch checkpoints and updates `best_models.pkl` with the best epoch (by validation AUC).
- You can interrupt and resume later; already-trained folds do not need to be retrained.


In [ ]:
history_fold_0 = train_one_fold(0)


### Train fold 1

Run this cell to train **fold 1**.

Tips:
- Adjust `EPOCHS`, `BATCH_SIZE`, and `LR` in **Imports & Global Settings**.
- The notebook saves all epoch checkpoints and updates `best_models.pkl` with the best epoch (by validation AUC).
- You can interrupt and resume later; already-trained folds do not need to be retrained.


In [ ]:
history_fold_1 = train_one_fold(1)


### Train fold 2

Run this cell to train **fold 2**.

Tips:
- Adjust `EPOCHS`, `BATCH_SIZE`, and `LR` in **Imports & Global Settings**.
- The notebook saves all epoch checkpoints and updates `best_models.pkl` with the best epoch (by validation AUC).
- You can interrupt and resume later; already-trained folds do not need to be retrained.


In [ ]:
history_fold_2 = train_one_fold(2)


### Train fold 3

Run this cell to train **fold 3**.

Tips:
- Adjust `EPOCHS`, `BATCH_SIZE`, and `LR` in **Imports & Global Settings**.
- The notebook saves all epoch checkpoints and updates `best_models.pkl` with the best epoch (by validation AUC).
- You can interrupt and resume later; already-trained folds do not need to be retrained.


In [ ]:
history_fold_3 = train_one_fold(3)


### Train fold 4

Run this cell to train **fold 4**.

Tips:
- Adjust `EPOCHS`, `BATCH_SIZE`, and `LR` in **Imports & Global Settings**.
- The notebook saves all epoch checkpoints and updates `best_models.pkl` with the best epoch (by validation AUC).
- You can interrupt and resume later; already-trained folds do not need to be retrained.


In [ ]:
history_fold_4 = train_one_fold(4)


## Isotonic Regression (OOF) + TEST evaluation/plots (AUC, Calibration, DCA)

This section performs probability calibration and evaluation:

1. Load the best checkpoint for each fold (from `best_models.pkl`, or by scanning `checkpoint_folder` as a fallback).
2. For each fold:
   - predict on the fold validation split and the test set
   - fit **Isotonic Regression** on the validation predictions
   - apply the calibrator to the fold validation (OOF) and to the test set
3. Aggregate calibrated test predictions by **mean ensembling across folds**.

Outputs:
- TEST ROC curve + AUC (with stratified bootstrap 95% CI)
- Calibration curve (reliability plot)
- Decision Curve Analysis (net benefit)

If you already have trained models, you may skip the fold training cells and run only this section (make sure `checkpoint_folder` and `best_models.pkl` point to your checkpoints).


In [ ]:
# =========================================================
# 6) Isotonic Regression (fit on each fold's val) + TEST evaluation
#    - uses the SAME `folds` indices as training
# =========================================================

def sanitize_prob(p, name="", clip01=True):
    p = np.asarray(p, dtype=np.float64).ravel()
    bad = ~np.isfinite(p)
    if bad.any():
        print(f"[sanitize_prob] {name}: {bad.sum()} non-finite values -> replaced with 0.5")
        p[bad] = 0.5
    if clip01:
        p = np.clip(p, 1e-6, 1 - 1e-6)
    return p

def bootstrap_auc_ci(y_true, y_prob, n_boot=2000, alpha=0.05, seed=42):
    y_true = np.asarray(y_true).astype(int).ravel()
    y_prob = sanitize_prob(y_prob, "bootstrap_auc_ci", clip01=True)

    rng = np.random.default_rng(seed)
    aucs = []
    pos = np.where(y_true == 1)[0]
    neg = np.where(y_true == 0)[0]
    if len(pos) == 0 or len(neg) == 0:
        return np.nan, (np.nan, np.nan)

    for _ in range(n_boot):
        samp_pos = rng.choice(pos, size=len(pos), replace=True)
        samp_neg = rng.choice(neg, size=len(neg), replace=True)
        idx = np.concatenate([samp_pos, samp_neg])
        rng.shuffle(idx)
        try:
            aucs.append(roc_auc_score(y_true[idx], y_prob[idx]))
        except Exception:
            pass

    aucs = np.array(aucs, dtype=float)
    auc_mean = float(np.mean(aucs))
    lo = float(np.quantile(aucs, alpha / 2))
    hi = float(np.quantile(aucs, 1 - alpha / 2))
    return auc_mean, (lo, hi)

def plot_calibration(y_true, y_prob, n_bins=10, strategy="quantile", title="Calibration (TEST)"):
    y_true = np.asarray(y_true).astype(int).ravel()
    y_prob = sanitize_prob(y_prob, "plot_calibration", clip01=True)
    frac_pos, mean_pred = calibration_curve(y_true, y_prob, n_bins=n_bins, strategy=strategy)

    plt.figure(figsize=(6, 6))
    plt.plot([0, 1], [0, 1], linestyle="--", linewidth=1, label="Ideal")
    plt.plot(mean_pred, frac_pos, marker="o", linewidth=2, label="Isotonic (TEST)")
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.xlabel("Mean predicted probability")
    plt.ylabel("Fraction of positives")
    plt.title(title)
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.legend(loc="best")
    plt.tight_layout()
    plt.show()

def plot_dca(y_true, y_prob, title="DCA (TEST)"):
    y_true = np.asarray(y_true).astype(int).ravel()
    y_prob = sanitize_prob(y_prob, "plot_dca", clip01=True)

    thresholds = np.linspace(0.01, 0.99, 100)
    n = len(y_true)
    prevalence = np.mean(y_true)

    nb_model = []
    nb_all = []
    for pt in thresholds:
        y_hat = (y_prob >= pt).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_hat).ravel()
        w = pt / (1 - pt)
        nb_m = (tp / n) - (fp / n) * w
        nb_a = prevalence - (1 - prevalence) * w
        nb_model.append(nb_m)
        nb_all.append(nb_a)

    nb_model = np.array(nb_model)
    nb_all = np.array(nb_all)
    nb_none = np.zeros_like(nb_model)

    plt.figure(figsize=(7, 5))
    plt.plot(thresholds, nb_model, linewidth=2, label="Isotonic (TEST)")
    plt.plot(thresholds, nb_all, linestyle=":", label="Treat All")
    plt.plot(thresholds, nb_none, linestyle="--", label="Treat None")

    plt.xlim(0, 1)
    plt.ylim(-0.1, max(np.max(nb_model), np.max(nb_all)) + 0.05)
    plt.xlabel("Threshold Probability")
    plt.ylabel("Net Benefit")
    plt.title(title)
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.legend(loc="best")
    plt.tight_layout()
    plt.show()

# ---------------------------------------------------------
# Load best model paths (from best_models.pkl) and map fold->path
# ---------------------------------------------------------
print("\n[Eval] Loading model list...")
if os.path.exists(best_models_file):
    with open(best_models_file, "rb") as f:
        best_models_paths = pickle.load(f)
else:
    best_models_paths = []

# fallback: search folder
if isinstance(best_models_paths, list) and len(best_models_paths) == 0:
    print("  [Warning] best_models.pkl not found/empty. Searching for .keras files...")
    best_models_paths = sorted(glob.glob(os.path.join(checkpoint_folder, "*.keras")))

if isinstance(best_models_paths, dict):
    best_models_paths = [best_models_paths.get(i, None) for i in range(NUM_FOLDS)]

print("best_models_paths:", best_models_paths)

fold_to_model = {}
for p in best_models_paths:
    if not p:
        continue
    m = re.search(r"fold_(\d+)", os.path.basename(p))
    if m:
        fold_to_model[int(m.group(1))] = p

def get_model_path_for_fold(fold: int) -> str:
    mp = fold_to_model.get(fold, None)
    if mp is None:
        raise FileNotFoundError(f"Model for fold={fold} not found. Train that fold first.")
    return mp

# ---------------------------------------------------------
# Predict per fold: fit isotonic on val, apply to val (OOF) and test
# ---------------------------------------------------------
print("\n[Eval] Predicting and applying Isotonic (fit on val, apply to TEST)...")
n_tv = len(y_train_val)
oof_iso = np.zeros(n_tv, dtype=np.float32)
test_iso_list = []

for fold in range(NUM_FOLDS):
    print(f"  Processing Fold {fold+1}/{NUM_FOLDS}...")
    model_path = get_model_path_for_fold(fold)
    print("    Model:", model_path)

    train_idx, val_idx = folds[fold]

    tf.keras.backend.clear_session()
    model = tf.keras.models.load_model(model_path, compile=False)

    val_prob  = model.predict([x_train_val_table[val_idx], x_train_val_image[val_idx]], batch_size=BATCH_SIZE, verbose=0).ravel()
    test_prob = model.predict([x_test_table, x_test_image], batch_size=BATCH_SIZE, verbose=0).ravel()
    val_prob  = sanitize_prob(val_prob, f"val_prob_fold{fold}")
    test_prob = sanitize_prob(test_prob, f"test_prob_fold{fold}")

    iso = IsotonicRegression(out_of_bounds="clip")
    iso.fit(val_prob, y_train_val[val_idx])

    val_prob_iso  = sanitize_prob(iso.predict(val_prob),  f"val_prob_iso_fold{fold}")
    test_prob_iso = sanitize_prob(iso.predict(test_prob), f"test_prob_iso_fold{fold}")

    oof_iso[val_idx] = val_prob_iso
    test_iso_list.append(test_prob_iso.astype(np.float32))

    del model
    gc.collect()

test_prob_iso_ens = np.mean(np.stack(test_iso_list, axis=0), axis=0)

# ---------------------------------------------------------
# Metrics & plots
# ---------------------------------------------------------
auc_test = roc_auc_score(y_test, test_prob_iso_ens)
auc_mean, (auc_lo, auc_hi) = bootstrap_auc_ci(y_test, test_prob_iso_ens, n_boot=AUC_N_BOOT, alpha=CI_ALPHA, seed=SEED)

print("\n[Eval] TEST AUC (Isotonic ensemble):", f"{auc_test:.4f}")
print("[Eval] TEST AUC bootstrap mean & 95%CI:", f"{auc_mean:.4f} ({auc_lo:.4f} - {auc_hi:.4f})")

fpr, tpr, _ = roc_curve(y_test, test_prob_iso_ens)
plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, linewidth=2, label=f"Isotonic (AUC={auc_test:.3f}, 95% CI {auc_lo:.3f}-{auc_hi:.3f})")
plt.plot([0, 1], [0, 1], linestyle="--", linewidth=1, label="Chance")
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC curve (TEST)")
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend(loc="best")
plt.tight_layout()
plt.show()

plot_calibration(y_test, test_prob_iso_ens, n_bins=CALIB_N_BINS, strategy=CALIB_STRATEGY,
                 title="Calibration curve (TEST) - Isotonic ensemble")
plot_dca(y_test, test_prob_iso_ens, title="Decision Curve Analysis (TEST) - Isotonic ensemble")
